In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

## QUAL A IDEIA
+ Entrar no site
+ Pesquisar por Notebook
+ Pegar todos os resultados da primeira página
+ Abrir cada resultado e extrair:
    1. Nome
    2. Preço a vista
    3. Preço a prazo

In [38]:
# Americanas

americanas_predf = {
    "name": [],
    "debit_price": [],
    "credit_price": [],
    "descount": []
}

driver = webdriver.Chrome()
wait = WebDriverWait(driver, poll_frequency=2, timeout=5)

sc = "notebook intel i5 RAM 4gb".replace(" ", "-")
rc = "notebook intel i5 RAM 4gb".replace(" ", "+")

driver.get(f"https://www.americanas.com.br/busca/{sc}?rc={rc}")

element = driver.find_element(By.XPATH, '//*[@id="rsyswpsdk"]/div/main/div/div[3]')
wait.until(lambda a: element.is_displayed())

elms = element.find_elements(By.CLASS_NAME, 'iRvjrG')

for elm in elms:
    try:
        name = elm.find_element(By.CLASS_NAME, 'gUjFDF').get_attribute('innerHTML')
        debit_price = elm.find_element(By.CLASS_NAME, 'JbUli').get_attribute('innerHTML').replace("R$ ", "").replace(".", "").replace(",", ".")
        credit_price = elm.find_element(By.CLASS_NAME, 'liXDNM').get_attribute('innerHTML').replace("R$ ", "").replace(".", "").replace(",", "."),
        
        try:
            debit_price = float(debit_price)
        except:
            print("debit_price", debit_price)
            
        try:
            credit_price = float(credit_price[0])
        except:
            print("credit_price", credit_price)
        
        americanas_predf["name"].append(name)
        americanas_predf["debit_price"].append(debit_price)
        americanas_predf["credit_price"].append(credit_price)
        
        if debit_price and credit_price:
            americanas_predf["descount"].append(100-(100*credit_price)/debit_price)
        else:
            americanas_predf["descount"].append(0)
    except:
        continue

driver.quit()

americanas_df = pd.DataFrame(americanas_predf)
americanas_df.rename(columns={"name": "NOME", "debit_price": "A VISTA", "credit_price": "PARCELADO", "descount": "DESC (%)"}, inplace=True)
americanas_df.head()

,NOME,A VISTA,PARCELADO,DESC (%)
0,Notebook asus Vivobook 15 X1500EA-EJ3671 Intel...,3299.0,2804.15,15.0
1,Notebook asus Vivobook 15 X1500EA-EJ3670W Inte...,3349.0,2846.65,15.0
2,Notebook asus Vivobook 15 X1500EA-EJ3668W Inte...,2899.0,2464.15,15.0
3,Notebook asus Vivobook 15 X1500EA-EJ3669 Intel...,2999.0,2549.15,15.0
4,Notebook asus Vivobook 15 X1500EP-EJ723W Intel...,4999.0,4249.15,15.0


In [40]:
#KaBuM!

kabum_predf = {
    "name": [],
    "debit_price": [],
    "credit_price": [],
    "descount": []
}

driver = webdriver.Chrome()
wait = WebDriverWait(driver, poll_frequency=2, timeout=5)

sc = "notebook intel i5 RAM 4gb".replace(" ", "-")

driver.get(f"https://www.kabum.com.br/busca/{sc}")

element = driver.find_element(By.XPATH, '//*[@id="listing"]/div[3]/div/div/div[2]/div[1]/main')
wait.until(lambda a: element.is_displayed())

elms = element.find_elements(By.CLASS_NAME, 'productCard')

for elm in elms:
    try:
        name = elm.find_element(By.CLASS_NAME, "kWpNqb").get_attribute("innerHTML")
        debit_price = elm.find_element(By.CLASS_NAME, "kLuRbO").get_attribute("innerHTML").split(";")[-1].strip()
        credit_price = elm.find_element(By.CLASS_NAME, "chPrxA").get_attribute("innerHTML").split(";")[-1].strip()
        
        debit_price = debit_price.replace("R$ ", "").replace(".", "").replace(",", ".")
        credit_price = credit_price.replace("R$ ", "").replace(".", "").replace(",", ".")
        
        try:
            debit_price = float(debit_price)
        except:
            print("debit_price", debit_price)
            
        try:
            credit_price = float(credit_price)
        except:
            print("credit_price", credit_price)
        
        kabum_predf["name"].append(name)
        kabum_predf["debit_price"].append(debit_price)
        kabum_predf["credit_price"].append(credit_price)        
        
        if debit_price and credit_price:
            kabum_predf["descount"].append(100-(100*credit_price)/debit_price)
        else:
            kabum_predf["descount"].append(0)
    except Exception as e:
        print(e)
        break

driver.quit()

kabum_df = pd.DataFrame(kabum_predf)
kabum_df.rename(columns={"name": "NOME", "debit_price": "A VISTA", "credit_price": "PARCELADO", "descount": "DESC (%)"}, inplace=True)
kabum_df.head()

debit_price 
debit_price 
debit_price 
debit_price 
debit_price 
debit_price 
debit_price 
debit_price 
debit_price 
debit_price 
debit_price 
debit_price 
debit_price 


,NOME,A VISTA,PARCELADO,DESC (%)
0,Notebook Gamer Lenovo Gaming 3i Intel Core i5-...,5227.26,3749.99,28.260886
1,Notebook Acer Swift 3 Evo Intel Core i5-1135G7...,,4299.99,0.000000
2,"Notebook Samsung Book Intel Core i5-1135G7, 8G...",3166.66,2699.10,14.765084
3,Notebook Gamer Acer NITRO 5 Intel Core i5-1140...,5789.46,4899.99,15.363609
4,Notebook Gamer Acer NITRO 5 Intel Core i5-1245...,7157.88,5299.99,25.955870


In [43]:
americanas_df.sort_values(by="DESC (%)", inplace=True)
kabum_df.sort_values(by="DESC (%)", inplace=True)

americanas_df.reset_index(drop=True, inplace=True)
kabum_df.reset_index(drop=True, inplace=True)

In [46]:
with pd.ExcelWriter("precos.xlsx", mode="w") as writer:
    # AMERICANAS
    americanas_df.to_excel(writer, sheet_name="AMERICANAS", index=False)
    
    column_length = max(americanas_df["NOME"].astype(str).map(len).max(), len("NOME"))
    writer.sheets['AMERICANAS'].column_dimensions["A"].width = column_length
    
    column_length = max(americanas_df["A VISTA"].astype(str).map(len).max(), len("A VISTA"))
    writer.sheets['AMERICANAS'].column_dimensions["A"].width = column_length
    
    column_length = max(americanas_df["PARCELADO"].astype(str).map(len).max(), len("PARCELADO"))
    writer.sheets['AMERICANAS'].column_dimensions["A"].width = column_length
    
    column_length = max(americanas_df["DESC (%)"].astype(str).map(len).max(), len("DESC (%)"))
    writer.sheets['AMERICANAS'].column_dimensions["A"].width = column_length
    
    # KABUM
    kabum_df.to_excel(writer, sheet_name="KaBuM!", index=False)